# Base Riscy Notebook

This notebook is supposed to exemplify how to set up and launch the execution of your core in the FPGA.

It is recommended to make a duplicate of this notebook that you can edit and change all the paths/names from `base_riscy` to `riscy` so you can always keep the initial version of both the overlay and the control of the core in a working state.

## Initialization of the notebook

A series of packages need to be imported, as well as defining a few variables and functions

In [1]:
from pynq import Overlay
from pynq import PL
from pynq import MMIO

zynq_system = Overlay("/home/xilinx/jupyter_notebooks/riscy/overlays/base_riscy.bit", download=False)

ins_mem = MMIO(0x40000000, 0x8000)
data_mem = MMIO(0x42000000, 0x8000)
riscv_control = MMIO(0x40008000, 0x1000)

def parse_coe_file(file_path):
    data_values = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        start_parsing = False
        
        for line in lines:
            line = line.strip()
            
            if start_parsing:
                values = line.split(',')
                data_values.extend([v.strip() for v in values if v.strip()])
            
            if "memory_initialization_vector=" in line:
                start_parsing = True
                line = line.split("=")[1]  # Get the values after '='
                values = line.split(',')
                data_values.extend([v.strip() for v in values if v.strip()])
                print(data_values)
    
    return data_values

def parse_and_process_file(filename, write_func):
    data_values = parse_coe_file(filename)
    count = 0
    offset = 0x0
    
    #print(f"Length of memory file: {len(data_values)} x 32-bits")
    while count < len(data_values):
        write_func(offset, int(data_values[count], 16))
        #print(f"Write {hex(offset)}: {hex(int(data_values[count], 16))}")
        offset += 0x4
        count += 1

ModuleNotFoundError: No module named 'pynq'

### Downloading the FPGA image (bitstream)

Now we will download the bistream into the PL of the FPGA:

In [ ]:
zynq_system.download()

## Control of the RISCY Core

Two top level pins of the RISCY core are connected to a control block (reboot_riscv_0 in the bd).

This block has a base address of `0x40008000`, but the relative offset of the 32-bit register controlling the connected riscv instance pins is `0x10`.
This 32-bit register is connected as follows:

* bit 4 = fetch_enable port
* bit 0 = reboot port

The default value of the control register is: `0x00000000`.

### Rebooting the RISCY Core

We will start by asserting high and deasserting the register bit that is connected to the reboot pin of the riscv core:

In [ ]:
data = riscv_control.read(0x10)
print(f"Read: {hex(data)}")
riscv_control.write(0x10, 0x00000001)
data = riscv_control.read(0x10)
print(f"Read: {hex(data)}")
riscv_control.write(0x10, 0x00000000)
data = riscv_control.read(0x10)
print(f"Read: {hex(data)}")

### Programming the instruction memory

Now that the riscv core is rebooted and stalling we can proceed to write our desired initialization sequences in the instruction memory (blk_mem_gen_1):

In [ ]:
parse_and_process_file("/home/xilinx/jupyter_notebooks/riscy/mem_files/program.coe", ins_mem.write_reg)

In [ ]:
# We can also read the instruction memory:
data=0x0
addr=0x0
for i in range(20):
    data = ins_mem.read(addr)
    print(f"Address is: {hex(addr)}; Value is: {hex(data)}.")
    addr += 0x4

### Programming the data memory

Same as before with the data memory (blk_mem_gen_2 in the bd):

In [ ]:
parse_and_process_file("/home/xilinx/jupyter_notebooks/riscy/mem_files/data.coe", data_mem.write_reg)

In [ ]:
# We can also read the data memory:
data=0x0
addr=0x0
for i in range(20):
    data = data_mem.read(addr)
    print(f"Address is: {hex(addr)}; Value is: {hex(data)}.")
    addr += 0x4

### Starting the execution of the RISCY Core

Now we can proceed to assert high to the fetch enable pin of the RISCY core:

In [ ]:
data = riscv_control.read(0x10)
print(f"Read: {hex(data)}")
riscv_control.write(0x10, 0x0000_0010)
data = riscv_control.read(0x10)
print(f"Read: {hex(data)}")

### Check the results of the execution of the core

TBD